In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
my_directory ='/content/drive/MyDrive/AI Projects/Resnet Chest Xray/Data'

In [4]:
my_directory

'/content/drive/MyDrive/AI Projects/Resnet Chest Xray/Data'

In [5]:
import os

In [6]:
os.listdir(my_directory)

['3', '2', '0', '1']

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
image_gen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [9]:
train_gen= image_gen.flow_from_directory(batch_size=40,
                                         directory=my_directory,
                                         shuffle=True,
                                         target_size=(256,256),
                                         class_mode='categorical',
                                         subset='training')
                                         

Found 428 images belonging to 4 classes.


In [10]:
val_gen= image_gen.flow_from_directory(batch_size=40,
                                         directory=my_directory,shuffle=True,
                                         target_size=(256,256),
                                         class_mode='categorical',
                                         subset='validation')

Found 104 images belonging to 4 classes.


In [11]:
train_images, train_labels = next(train_gen)

In [12]:
train_images.shape

(40, 256, 256, 3)

In [13]:
train_labels.shape

(40, 4)

In [14]:
train_labels[0:6]

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [15]:
label_names= {0:'corona',1:'healthy',2:'viral',3:'bacterial'}

In [16]:
label_names

{0: 'corona', 1: 'healthy', 2: 'viral', 3: 'bacterial'}

In [17]:
from tensorflow.keras.applications.resnet50 import ResNet50


In [18]:
from tensorflow.keras.layers import Input
my_resnet = ResNet50(weights='imagenet',include_top= False,input_tensor= Input(shape= (256,256,3)))


In [19]:
my_resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

## All layers except the last 15 layers are frozen

In [20]:
for layer in my_resnet.layers[:-15]:
  layer.trainable=False

#### Check the trainable parameter after and before freezing

In [21]:
my_resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Add, Dense, Flatten, Conv2D,  Activation, AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#### Start and Make the mail model as everything that is inside the my_resnet model and then start building on it

In [23]:
inputs = my_resnet.output

In [24]:
h= AveragePooling2D(pool_size=(4,4))(inputs)
h= Flatten(name='flatten')(h)
h= Dense(256,activation='relu')(h)
h= Dropout(0.3)(h)
h= Dense(128,activation='relu')(h)
h= Dropout(0.2)(h)
outputs= Dense(4,activation='softmax')(h)



In [25]:
model= Model(inputs= my_resnet.input,outputs=outputs)

In [26]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [27]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [28]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [29]:
earlystopping = EarlyStopping(monitor='val_loss',patience=22,mode='auto')

checkpointer = ModelCheckpoint(filepath='weights.hdf5',verbose=1,save_best_only=True)

In [30]:
train_generator= image_gen.flow_from_directory(batch_size=4,
                                         directory=my_directory,
                                         shuffle=True,
                                         target_size=(256,256),
                                         class_mode='categorical',
                                         subset='training')
                                         

Found 428 images belonging to 4 classes.


In [31]:
val_generator= image_gen.flow_from_directory(batch_size=4,
                                         directory=my_directory,
                                         shuffle=True,
                                         target_size=(256,256),
                                         class_mode='categorical',
                                         subset='validation')

Found 104 images belonging to 4 classes.


## You can set steps_per_epoch equal to num_samples // batch_size, which is a typical choice.

In [32]:
size_of_batch=15

In [ ]:
history= model.fit_generator(train_generator,
                             steps_per_epoch= len(train_generator)//size_of_batch,
                             epochs=100,
                             validation_data=val_generator,
                             callbacks=[earlystopping,checkpointer])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
7/7 [==============================] - ETA: 0s - loss: 2.8033 - accuracy: 0.2857
Epoch 00001: val_loss improved from inf to 3.98725, saving model to weights.hdf5
7/7 [==============================] - 29s 4s/step - loss: 2.8033 - accuracy: 0.2857 - val_loss: 3.9872 - val_accuracy: 0.2500
Epoch 2/100
7/7 [==============================] - ETA: 0s - loss: 2.0588 - accuracy: 0.2143
Epoch 00002: val_loss did not improve from 3.98725
7/7 [==============================] - 27s 4s/step - loss: 2.0588 - accuracy: 0.2143 - val_loss: 6.1982 - val_accuracy: 0.2500
Epoch 3/100
7/7 [==============================] - ETA: 0s - loss: 2.0491 - accuracy: 0.2143
Epoch 00003: val_loss did not improve from 3.98725
7/7 [==============================] - 27s 4s/step - loss: 2.0491 - accuracy: 0.2143 - val_loss: 9.0422 - val_accuracy: 0.2500
Epoch 4/100
7/7 [==============================] - ETA: 0s - loss: 1.7561 - accur

In [ ]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
df =pd.DataFrame(history.history)
df.head(2)

In [ ]:
plt.plot(df.loss,label='Loss',c='r')
plt.plot(df.accuracy,label='Accuracy',c='b')
plt.legend();

In [ ]:
plt.plot(df.loss,label='Loss',c='r')
plt.legend();

In [ ]:
plt.plot(df.accuracy,label='Accuracy',c='b')
plt.legend();

In [ ]:
plt.plot(df.loss,label='Training Loss',c='r')
plt.plot(df.val_loss,label='Validation loss',c='b')
plt.legend();

In [ ]:
plt.plot(df.accuracy,label='Training Accuracy',c='r')
plt.plot(df.val_accuracy,label='Validation Accuracy',c='b')
plt.legend();

In [ ]:
test_directory ='/content/drive/MyDrive/AI Projects/Resnet Chest Xray/Test'

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator= test_gen.flow_from_directory(batch_size=40,
                                         directory=test_directory,
                                         shuffle=True,
                                         target_size=(256,256),
                                         class_mode='categorical',
                                         subset='training')

In [ ]:
test_generator.n

In [ ]:
results=model.evaluate(test_generator, 
                         steps= test_generator.n//size_of_batch,
                         verbose=1
)

In [ ]:
results[1]